In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()

from sklearn.model_selection import train_test_split 
import lightgbm as lgb

import cufflinks as cf
cf.go_offline()

import xgboost as xgb 


In [ ]:
#Load the tran and test dataframes 
application_train=
application_test=

In [ ]:
#Features dataframe using train and test set 
def featuresDF(df_train,df_test,target):
    training=df_train
    testing=df_test

    training['is_train']=1
    training['is_test']=0
    testing['is_train']=0
    testing['is_test']=1
    
    train_X=training.drop([target],axis=1)
    test_X=testing
    data=pd.concat([train_X,test_X],axis=0)
    return data
data=featuresDF(application_train,application_test,"Comment")

In [ ]:
#Target variable extraction 
def target(df_train,target):
    training=df_train
    Y=training[target]
    return Y
    
Y=target(application_train,'TARGET')

In [ ]:
#Missing values Analysis 
def missingValuesAnalysis(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
df=missingValuesAnalysis(application_train)

In [ ]:
#Function to capture numerical variables 
def _get_numerical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype!='object']
    return feats 
numFeatures=_get_numerical_features(data)

In [ ]:
#Function to capture categorical variables 
def _get_categorical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype=='object']
    return feats 
cats=_get_categorical_features(data)

In [ ]:
#Function to factorize categorical variables 
def _factorize_categoricals(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 
#data=_factorize_categoricals(data,cats)

In [ ]:
#Function to convert categorical variables into dummy variables 
def _get_dummies(df,cats):
    for col in cats:
         df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    return df
#data=_get_dummies(data,cats)

In [ ]:
#Doing factorize or dummy variable encoding 
def encoding(df,flag):
    data_cats = _get_categorical_features(df)
    if(flag==1):
        df = _factorize_categoricals(df,data_cats)
    else:
        df=_get_dummies(df,data_cats)
        df=df.drop(data_cats,axis=1)
    return df
data=encoding(data,0)

In [ ]:
#Missing values handling since XGBOOST  cannot handle missing values internally 
data1=data.iloc[:,:].values #This only contain values (not headers) . Objective of this is to convet data frame into matrix
#in order to aply imputation and standerdization. 

numInd=_get_numerical_features(data)
catInd=_get_categorical_features(data)

numImputer = Imputer(missing_values ='NaN', strategy='median', axis=0)
numImputer= numImputer.fit(data1[:,(numInd)])
data1[:,(numInd)]=numImputer.transform(data1[:,(numInd)])

catImputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis =0)
catImputer=catImputer.fit(data1[:,(catInd)])
data1[:,(catInd)]=catImputer.transform(data1[:,(catInd)])

#Standerdizing 
min_max_scaler = preprocessing.MinMaxScaler()
data1[:,(numInd)] = min_max_scaler.fit_transform(data1[:,(numInd)])

data=pd.DataFrame(data=data1[0:,0:],columns=data.columns.values)


In [ ]:
#Splitting into train and test set 
ignoreFeatures = ['CUST_ID', 'is_train', 'is_test']
importantFeatures = [col for col in data.columns if col not in ignoreFeatures]
trainX = data[data['is_train'] == 1][importantFeatures]
testX = data[data['is_test'] == 1][importantFeatures]
x_train, x_val, y_train, y_val = train_test_split(trainX, Y, test_size=0.2, random_state=18)

# Methos 1 with train and test split

In [ ]:
#Setting model parameters 
model = XGBClassifier()
model.set_params(n_estimators = 2000)
model.set_params(booster='gbtree')
model.set_params(objective = 'binary:logistic')
model.set_params(max_depth=9)
model.set_params(learning_rate= 0.01)
model.set_params(gamma= 1)
model.set_params(min_child_weight= 1)
model.set_params(colsample_bytree= 0.8)
model.set_params(subsample= 0.8 )
model.set_params(reg_alpha=1)
model.set_params(reg_alpha=1)
model.set_params(scale_pos_weight=1)

eval_set = [(x_train, y_train), (x_val, y_val)]
model=model.fit(x_train, y_train, eval_metric=["error","auc"], eval_set=eval_set, verbose=True)

In [ ]:
#Predict probabilities 
preds=model.predict_proba(testX)
preds
#Prediction 
preds = model.predict(testX)
test_id=application_test['CUST_ID']
sub_xgb = pd.DataFrame()
sub_xgb['CUST_ID'] = test_id
sub_xgb['COMMENT'] = preds
sub_xgb.to_csv("Prediction.csv", index=False)
sub_xgb.head()

In [ ]:
# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['auc'])
x_axis = range(0, epochs)

In [ ]:
# plot AUC from train and test set 
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['auc'], label='Train')
ax.plot(x_axis, results['validation_1']['auc'], label='Test')
ax.legend()
pyplot.ylabel('auc error')
pyplot.title('XGBoost auc')
pyplot.show()

In [ ]:
# plot classification error 
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
pyplot.ylabel('classification error')
pyplot.title('XGBoost classification error')
pyplot.show()

In [ ]:
#Model fitting with early stopping 
model = XGBClassifier()
model.set_params(n_estimators = 1000)
eval_set = [(x_train, y_train), (x_val, y_val)]
model=model.fit(x_train, y_train, early_stopping_rounds=10, eval_metric=["error","auc"], eval_set=eval_set, verbose=True)

# Method 2 with Cross validation (CV)

In [ ]:
# 5 fold cross validation is more accurate than using a single validation set
import random
SEED=3
random.seed(SEED)
np.random.seed(SEED)

features=trainX
labels=Y

cv_folds = 5
early_stopping_rounds = 50
model=XGBClassifier(seed = SEED )

model.set_params(n_estimators = 2000)
model.set_params(booster='gbtree')
model.set_params(objective = 'binary:logistic')
model.set_params(max_depth=9)
model.set_params(learning_rate= 0.01)
model.set_params(gamma= 1)
model.set_params(min_child_weight= 1)
model.set_params(colsample_bytree= 0.8)
model.set_params(subsample= 0.8 )
model.set_params(reg_alpha=1)
model.set_params(reg_alpha=1)
model.set_params(scale_pos_weight=1)

xgb_param = model.get_xgb_params()
xgtrain = xgb.DMatrix(features, label=labels)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round = 1000, nfold = cv_folds, metrics = 'auc', early_stopping_rounds = early_stopping_rounds, seed = SEED)

In [ ]:
xgb_param

# Tunning parameters  - Grid Search 

In [ ]:
#1. Tunning   max_depth and min_child weight 

from sklearn.grid_search import GridSearchCV

model.set_params(n_estimators = 80)
param_test1 = {
'max_depth': [6,7,8,9,10],
'min_child_weight': [1,2,3],
}
features=trainX
labels=Y
gsearch1 = GridSearchCV(estimator = model, param_grid = param_test1, scoring = 'roc_auc', n_jobs = 4, iid = False,verbose = 2)
gsearch1.fit(features,labels)
print(gsearch1.best_params_)
print(gsearch1.best_score_)

In [ ]:
#2 .Tunning gamma

model.set_params(n_estimators = 80)
model.set_params(max_depth = 3)
model.set_params(min_child_weight = 10)
param_test2 = {
'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = model, param_grid = param_test2, scoring = 'roc_auc',n_jobs = 4, iid = False, cv = cv_folds, verbose = 2)
gsearch2.fit(features,labels)
print gsearch2.best_params_
print gsearch2.best_score_

In [ ]:
#3. Gtting new optimal number of trees 

import random
SEED=3
random.seed(SEED)
np.random.seed(SEED)

features=trainX
labels=Y

cv_folds = 5
early_stopping_rounds = 50
model=XGBClassifier(seed = SEED )
xgb_param = model.get_xgb_params()
xgtrain = xgb.DMatrix(features, label=labels)
cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round = 1000, nfold = cv_folds, metrics = 'auc', early_stopping_rounds = early_stopping_rounds, seed = SEED)
print(cvresult)
print("Optimal number of trees (estimators) is %i" % csresult.shape[0])

In [ ]:
#4. Tunning subsample and colsample by tree 

model.set_params(n_estimators = 2000)
model.set_params(max_depth = 10)
model.set_params(min_child_weight = 3)
model.set_params(gamma = 1)
param_test3 = {
'subsample' : [i/10.0 for i in range(6,11)],
'colsample_bytree' : [i/10.0 for i in range(6,11)]
}
gsearch3 = GridSearchCV(estimator = model, param_grid = param_test3, scoring = 'roc_auc',n_jobs = 4, iid = False, cv =5, verbose = 2)
gsearch3.fit(features,labels)
print(gsearch3.best_params_)
print (gsearch3.best_score_)

In [ ]:
#5. Tunning regularization parameter 

model.set_params(n_estimators = 108)
model.set_params(max_depth = 2)
model.set_params(min_child_weight = 1)
model.set_params(subsample=0.9)
model.set_params(colsample_bytree=0.6)
param_test4 = {
'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch4 = GridSearchCV(estimator = model, param_grid = param_test4, scoring = 'roc_auc',n_jobs = 4, iid = False, cv = cv_folds, verbose = 2)
gsearch4.fit(features,labels)
print gsearch4.best_params_
print gsearch4.best_score_

In [ ]:
print(model.feature_importances_)
plot_importance(model)
pyplot.show()

In [ ]:
from xgboost import plot_tree
import graphviz
# plot single tree
#plot_tree(model)
#plt.show()
plot_tree(model, num_trees=0, rankdir='LR')

# Tunning parameters - Random Search 

In [ ]:
#Random search 
def randomsearch_xgboost(df):
        param_distributions={'max_depth': sp.stats.randint(1, 11),
                             'subsample': sp.stats.uniform(0.25, 0.75),
                             'colsample_bytree': sp.stats.uniform(0.25, 0.75)
        }
        xgb_model = XGBClassifier()
        rs = RandomizedSearchCV(xgb_model,
                                param_distributions,
                                cv=10,
                                n_iter=20,
                                scoring="log_loss",
                                n_jobs=1,
                                verbose=2)
        rs.fit(train_X, train_y.transpose()[0]) 
        predict = rs.predict_proba(test_X)
        return predict[:, 1]